每次操作都要按「執行」，也就是框框左邊的圓形按鈕

首先，請先按下設定區下面的按鈕，執行所有安裝設定

## 設定區

### 安裝套件

In [ ]:
!pip install Pillow numpy matplotlib librosa exif

### EOF 偵測

In [ ]:
def detect_EOF(file_path, num_bytes=256):
    """
    Reads the last few bytes of a file and displays them in a hex and ASCII format
    similar to the 'xxd' command.

    Args:
        file_path (str): The full path to the file.
        num_bytes (int): The number of bytes to read from the end of the file.
    """
    try:
        with open(file_path, 'rb') as f:
            f.seek(0, 2)  # Seek to the end of the file
            file_size = f.tell()
            f.seek(max(0, file_size - num_bytes), 0) # Seek back num_bytes from the end
            data = f.read(num_bytes)

        offset = max(0, file_size - num_bytes)
        bytes_per_line = 16

        for i in range(0, len(data), bytes_per_line):
            hex_part = ' '.join([f'{b:02x}' for b in data[i:i+bytes_per_line]])
            ascii_part = ''.join([chr(b) if 32 <= b <= 126 else '.' for b in data[i:i+bytes_per_line]])
            print(f'{offset + i:08x}: {hex_part:<{bytes_per_line * 3 - 1}} {ascii_part}')

    except FileNotFoundError:
        print(f"Error: The file '{file_path}' was not found.")
        print("Please make sure the file is uploaded to your Colab environment and the path is correct.")
    except Exception as e:
        print(f"An error occurred: {e}")

### LSB 偵測＋解密

In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

def detect_LSB(image_path):
    """
    Opens an image, extracts its LSB plane, and displays both images.

    Args:
        image_path (str): The full path to the image file within the Colab environment.
    """
    try:
        # For visualization, RGB is sufficient and standard.
        original_image = Image.open(image_path).convert('RGB')
        image_array = np.array(original_image)
        lsb_array = (image_array % 2) * 255
        lsb_image = Image.fromarray(lsb_array.astype('uint8'), 'RGB')

        fig, axes = plt.subplots(1, 2, figsize=(12, 6))

        axes[0].imshow(original_image)
        axes[0].set_title('Original Image')
        axes[0].axis('off')

        axes[1].imshow(lsb_image)
        axes[1].set_title('LSB Visualization')
        axes[1].axis('off')

        plt.tight_layout()
        plt.show()

    except FileNotFoundError:
        print(f"Error: The file '{image_path}' was not found.")
        print("Please make sure the file is uploaded to your Colab environment and the path is correct.")
    except Exception as e:
        print(f"An error occurred: {e}")

def decode_LSB(image_path):
    """
    Extracts a hidden message from an image's LSBs. If the message appears
    to be a CSV file, it saves it. Otherwise, it prints the content.

    Args:
        image_path (str): The path to the steganographic image file.
    """
    try:
        # --- FIX: Convert to RGBA to match the web tool's encoding process ---
        # The web encoder uses all 4 channels (R,G,B,A) to store data.
        image = Image.open(image_path).convert('RGBA')
        image_array = np.array(image)

        binary_data = image_array.flatten() % 2
        byte_data = np.packbits(binary_data)

        # --- FIX: Decode using 'latin-1' for robustness. ---
        # This encoding maps every possible byte value (0-255) to a character,
        # preventing decoding errors that can happen with UTF-8 on raw binary data.
        # This allows us to reliably find our text-based delimiter.
        full_message = byte_data.tobytes().decode('latin-1')

        delimiter = "|||||"
        delimiter_index = full_message.find(delimiter)

        if delimiter_index != -1:
            revealed_message = full_message[:delimiter_index]

            # Heuristic to check if the content is likely a CSV file
            if ',' in revealed_message and '\n' in revealed_message:
                output_filename = 'decoded_output.csv'
                try:
                    with open(output_filename, 'w', newline='', encoding='utf-8') as f:
                        f.write(revealed_message)
                    print(f"--- CSV Content Detected ---")
                    print(f"Message saved to '{output_filename}' in your Colab session files.")
                    print("--------------------------")
                except Exception as e:
                    print(f"Could not save the CSV file. Error: {e}")
            else:
                # If not a CSV, just print the text content
                print("--- Message Found ---")
                print(revealed_message)
                print("---------------------")
        else:
            print("No message with a valid delimiter found.")
            print("\n--- Partial Data (Fallback) ---")
            print(full_message[:200])
            print("-------------------------------")

    except FileNotFoundError:
        print(f"Error: The file '{image_path}' was not found.")
    except Exception as e:
        print(f"An error occurred during decoding: {e}")

### 培根解密

In [ ]:
import csv
import time # Import the time module

def decode_bacon(file_name, A_char, B_char):
  try:
    with open(file_name, 'r', encoding='utf-8') as file:
        content = file.read()

    ## 1. Extract and Group the A/B Sequence
    # ----------------------------------------
    print("## Task 1: Extracting A/B Sequence ##\n")
    print("Extracting characters...") # Progress indicator
    time.sleep(1) # Add a small delay

    ab_sequence = ""
    for char in content:
        if char == A_char:
            ab_sequence += "A"
        elif char == B_char:
            ab_sequence += "B"

    # Group the sequence into chunks of 5 characters
    grouped_sequence = [ab_sequence[i:i+5] for i in range(0, len(ab_sequence), 5)]

    print("Extracted A/B Sequence (in groups of 5):")
    print(' '.join(grouped_sequence))
    print("-" * 40)
    time.sleep(1) # Add a small delay


    ## 2. Create the Mapping CSV File
    # ---------------------------------
    print("\n## Task 2: Reading Mapping File ##\n")
    print("Reading mapping file...") # Progress indicator
    time.sleep(1) # Add a small delay

    bacon_map = {}
    csv_path = './decoded_output.csv'
    try:
        with open(csv_path, mode='r', encoding='utf-8') as csv_file:
            reader = csv.reader(csv_file)
            for row in reader:
                if len(row) == 2:
                    bacon_map[row[1]] = row[0]
        print(f"Baconian cipher mapping has been read from: {csv_path}")
        print("-" * 40)
        time.sleep(1) # Add a small delay
    except FileNotFoundError:
        print(f"Error: The mapping file '{csv_path}' was not found.")
        print("Please make sure 'decoded_output.csv' exists in your Colab session files.")
        return # Exit if the mapping file is not found
    except Exception as e:
        print(f"An error occurred while reading the mapping file: {e}")
        return # Exit on other file reading errors


    ## 3. Decode and Display the Message
    # -----------------------------------
    print("\n## Task 3: Decoding the Message ##\n")
    print("Decoding message...") # Progress indicator
    time.sleep(1) # Add a small delay


    decoded_message = ""
    for group in grouped_sequence:
        # Ensure the group is a valid 5-character key
        if group in bacon_map:
            decoded_message += bacon_map[group]
        else:
            decoded_message += "?" # Add a placeholder for unknown sequences

    print("🎉 The decoded secret message is:")
    print(decoded_message.upper())

  except FileNotFoundError:
      print(f"Error: The input file '{file_name}' was not found.")
      print("Please make sure the file is uploaded to your Colab environment and the path is correct.")
  except Exception as e:
      print(f"An error occurred during Baconian decoding: {e}")

### 頻譜解密

In [ ]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import Audio, display # Import Audio and display

def decode_spectrogram(audio_path):
  try:
    # Load file
    y, sr = librosa.load(audio_path)

    # Play the audio file
    print("Playing audio...")
    display(Audio(y, rate=sr))

    # Compute the short-time Fourier transform (STFT)
    D = librosa.stft(y)

    # Convert to magnitude
    S_db = librosa.amplitude_to_db(np.abs(D), ref=np.max)

    # Plot with log-frequency scale
    plt.figure(figsize=(10, 5))
    librosa.display.specshow(S_db, sr=sr, x_axis='time', y_axis='log')
    plt.colorbar(format='%+2.0f dB')
    plt.title('Spectrogram')
    plt.tight_layout()
    plt.show()
  except FileNotFoundError:
    print(f"Error: The audio file '{audio_path}' was not found.")
    print("Please make sure the file is uploaded to your Colab environment and the path is correct.")
  except Exception as e:
    print(f"An error occurred during spectrogram decoding: {e}")

### metadata 解密

In [ ]:
import exif

def decode_meta(file_path):
    """
    Extracts metadata from an image file using the exif package.

    Args:
        file_path (str): The path to the image file.
    """
    try:
        with open(file_path, 'rb') as f:
            img = exif.Image(f)

        if img.has_exif:
            print(f"Metadata for {file_path}:")
            if 'user_comment' in img.list_all():
                print("User Comment:")
                print(img.get('user_comment'))
            else:
                print("No user comment found.")
            #for tag in img.list_all():
                # You can filter for specific tags if needed, e.g., 'user_comment'
                # if 'user' in tag.lower():
                #print(f"{tag}: {img.get(tag)}")
        else:
            print(f"No EXIF metadata found in {file_path}.")

    except FileNotFoundError:
        print(f"Error: The file '{file_path}' was not found.")
        print("Please make sure the file is uploaded to your Colab environment and the path is correct.")
    except Exception as e:
        print(f"An error occurred during metadata decoding: {e}")

# 隱寫術操作

把檔案上傳到左邊的視窗

在你想處理的檔案上方按右鍵後選擇「複製路徑」

把路徑貼在單引號 '' 的裡面

範例：



```
file = '/content/example.jpg'
```



In [ ]:
file = ''

貼上後記得要執行喔

## 隱寫偵測

### EOF 偵測

In [ ]:
detect_EOF(file)

### LSB 偵測

In [ ]:
detect_LSB(file)

## 隱寫解碼

### LSB 解碼

In [ ]:
decode_LSB(file)

### 培根解碼
請先輸入 A 和 B 分別是什麼
例如：

```
A = '1'
B = '2'
```



In [ ]:
A = ''
B = ''
decode_bacon(file, A, B)

### 頻譜解碼

In [ ]:
decode_spectrogram(file)

### metadata 解碼

In [ ]:
decode_meta(file)